In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from cvxopt import matrix
import cvxopt.solvers
from SVM_cvxopt import SVMModel

def linearKernel(x1, x2):
    return np.inner(x1, x2)

def polynomialKernel(x1, x2, p):
    return (1 + np.inner(x1, x2))**p

def guassianKernel(x1, x2, sigma = 5.0):
    return np.exp(-np.linalg.norm(x1 - x2)**2 / (2*sigma))
   
    
class SVMModel(object):
    def __init__(self, kernel, x, y, testdata, p = None, C = 1):
        self.kernel = kernel
        if self.kernel == polynomialKernel:
            assert p is not None
            
        self.p = p   
        self.x = x
        self.y = y
        self.testdata = testdata
        self.C = C
        
        self.weights_and_bias(self.x, self.y)
        self.pos = self.test_label(testdata)
        
    def fit(self, x, y):
        """
            Calculate the convext optimization problem
        Args:
            x: training dataset, shape if [num_samples, num_features]
            y: data for different class, shape is [1, num_samples]
        """
        n_samples, n_features = x.shape
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                if self.kernel == linearKernel:
                    K[i, j] = self.kernel(x[i], x[j])
                elif self.kernel == polynomialKernel:
                    K[i, j] = self.kernel(x[i], x[j], self.p)
                    
        # loss function 
        p = matrix(np.outer(y, y) * K)
        q = matrix(-1 * np.ones((n_samples, 1)), tc = 'd')
        # contrain condition 0 <= alpha <= C
        G = matrix(np.vstack((-np.eye(n_samples), np.identity(n_samples))), tc = 'd')
        A = matrix(y, (1, n_samples), tc = 'd')
        b = matrix(0.0)
        h1 = np.zeros((n_samples, 1))
        h2 = self.C * np.ones((n_samples, 1))
        h = matrix(np.vstack((h1, h2)), tc = 'd')
        solution = cvxopt.solvers.qp(p, q, G, h, A, b)
        
        return solution, np.ravel(solution["x"]), K
    
    def weights_and_bias(self, x, y):
        """
            Calculate the line weights and bias
        Args:
            x: training dataset, shape is [num_samples, num_features]
            y: data for different class, shape is [1, num_samples]
        """
        n_samples, n_features = x.shape
        _, solution, K = self.fit(x, y)  
        print("K.shape is {}".format(K.shape))
        
        b = 0       
        if self.kernel == linearKernel:  
            index = np.where(solution > 0)
            index = index[0]
            self.alpha = solution[index]
            self.data = x[index]
            self.label = y[index]
            
            self.w = np.zeros((1, n_features))
            for al, d, l in zip(self.alpha, self.data, self.label):
                self.w += al * l * d
                
            for i in range(len(index)):
                b += self.label[i]
                b -= np.dot(self.w, self.data[i])
            self.b = b / float(len(index))
            
        else:
            index = np.where(solution > -1)
            index = index[0]
            self.alpha = solution[index]
            self.data = x[index]
            self.label = y[index]
            print("len(index) is {}".format(len(index)))
            print("self.label.shape is {}".format(self.label.shape))
            
            self.w = None
            for i in range(len(index)):
                b += self.label[i]
                data = K[index[i], index]
                temp = 0
                for al, d, l in zip(self.alpha, self.label, data):
                    temp += al * d * l
                b -= temp
            self.b = b / float(len(index))
            
        print("self.b is {}".format(self.b))
            
    def test_label(self, testdata):
        pos_class = []
        if self.w is not None:
            result = np.dot(testdata, self.w.T) + self.b
            pos_class.append(result)
        else:
            for i in range(testdata.shape[0]):
                s = 0
                for al, d, l in zip(self.alpha, self.data, self.label):
                    s += al * l * self.kernel(d, testdata[i], self.p)
                s += self.b
                pos_class.append(s) 
        pos_class = np.array(pos_class, dtype = "float")
        pos_class = np.squeeze(pos_class)
        return pos_class
           
            

ROOT = os.getcwd()
print("current path is {}".format(ROOT))
filedir = ROOT + "/att_faces"
dirlist = os.listdir(filedir)
data = []
label = []
for dirs in dirlist:
    imgdir = filedir + "/" + dirs
    imglabel = dirs[1:] 
    imglist = os.listdir(imgdir)
    for img in imglist:
        imgpath = imgdir + "/" + img
        imgdata = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        imgdata = imgdata.reshape((1, -1))
        data.append(imgdata)
        label.append(imglabel)

data = np.array(data, dtype = "float")
data = np.squeeze(data)
label = np.array(label, dtype = "float")
print("data.shape is {}".format(data.shape))
print("label.shape is {}".format(label.shape))
label_flat = np.ravel(label)
unique_label = np.unique(label_flat)
num_samples = data.shape[0]
num_class = len(unique_label)
print("num_samples is {}".format(num_samples))
print("num_class is {}".format(num_class))


traindata, testdata, trainlabel, testlabel = train_test_split(data, label, test_size = 0.2, random_state = 0) 
print("traindata.shape is {}, trainlabel.shape is {}".format(traindata.shape, trainlabel.shape))
print("testdata.shape is {}, testlabel is {}".format(testdata.shape, testlabel.shape))

# classify the dataset based on the labe,  it will construct a matrix with shape of (num_classes X num_samples)
# in 40 classes, when the data label is in this class, then the value is 1
one_hot_column = len(trainlabel)
one_hot = -1 * np.ones(shape = (num_class, one_hot_column), dtype = "float")
for i in range(num_class):
    for j in range(one_hot_column):
        if trainlabel[j] == (i+1):
            one_hot[i][j] = 1

print("---- check the one-hot matrix ----")
label_valid = np.argmax(one_hot, axis = 0)
print("True label is")
print(trainlabel)
print("label_valid is")
print(label_valid)

Label1 = one_hot[0]
tindex = np.where(trainlabel == 1.0)
print(tindex)
index = np.where(Label1 == 1)
print(index)
print(len(Label1))

pos_tabel = []
for i in range(40):
    label_i = one_hot[i]
    svm = SVMModel(polynomialKernel, traindata, label_i, testdata, p = 2, C = 1) 
    pos = svm.pos
    item = pos.reshape((80, -1))
    if i == 0:
        pos_table = item
    else:
        pos_table = np.concatenate([pos_table, item], axis = 1)
print(pos_table.shape)
print(pos_table[0, :])
pre_label = np.argmax(pos_table, axis = 1)
print("testlabel is {}".format(testlabel))
print("prelabel is {}".format(pre_label))

count = 0
for tl, pl in zip(testlabel, pre_label):
    if tl == pl + 1:
        count += 1
accuracy = count / testlabel.shape[0]
print("accuracy is {}".format(accuracy))